<a href="https://colab.research.google.com/github/advik-7/Deep_Learning_projects/blob/main/Basic_RAG_in_Kannada.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 7.9 MB/s eta 0:00:00


In [3]:
import faiss
import numpy as np
import time
from sentence_transformers import SentenceTransformer

def read_text_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.readlines()

def vectorize_text(text_data, model):
    return model.encode(text_data, convert_to_numpy=True)

def create_faiss_index(vectors):
    index = faiss.IndexFlatL2(vectors.shape[1])
    index.add(vectors)
    return index

def adjust_query_vector(query_vector, required_dim):
    current_dim = query_vector.shape[1]
    if current_dim == required_dim:
        return query_vector
    elif current_dim < required_dim:
        padding = np.zeros((query_vector.shape[0], required_dim - current_dim), dtype=np.float32)
        return np.hstack((query_vector, padding))
    else:
        return query_vector[:, :required_dim]

def query_faiss_index(index, query_vector, k):
    query_vector = np.array(query_vector, dtype=np.float32)
    if query_vector.ndim == 1:
        query_vector = query_vector.reshape(1, -1)
    distances, indices = index.search(query_vector, k)
    return distances, indices

def retrieve_documents_batch(index, query_vector, k, text_data):
    distances, indices = query_faiss_index(index, query_vector, k)
    batch = [(text_data[idx].strip(), dist) for idx, dist in zip(indices[0], distances[0])]
    return batch

def generate_augmented_output(query, retrieved_docs_batch):
    combined_documents = "\n".join([f"Document: '{doc}' (Distance: {distance:.4f})" for doc, distance in retrieved_docs_batch])
    response_content = " ".join([doc for doc, _ in retrieved_docs_batch])
    augmented_response = f"Based on the retrieved documents, the information provided suggests: {response_content}"
    output = (
        f"Query: '{query}'\n"
        f"Combined Retrieved Documents:\n{combined_documents}\n"
        f"Augmented Response: '{augmented_response}'\n"
    )
    yield output

if __name__ == "__main__":
    model = SentenceTransformer('bert-base-multilingual-cased')
    file_path = "/content/Kannada_RAG_practise.txt"
    text_data = read_text_file(file_path)
    vectors = vectorize_text(text_data, model)
    faiss_index = create_faiss_index(vectors)
    query_text = input("Enter a query text in Kannada: ")
    query_vector = vectorize_text([query_text], model)
    required_dim = vectors.shape[1]
    query_vector_adjusted = adjust_query_vector(query_vector, required_dim)
    k = 5
    start_time = time.time()
    retrieved_docs_batch = retrieve_documents_batch(faiss_index, query_vector_adjusted, k, text_data)
    print("\nGenerated augmented response:")
    for augmented_output in generate_augmented_output(query_text, retrieved_docs_batch):
        print(augmented_output)
    end_time = time.time()
    print(f"\nTime taken for retrieval and generation: {end_time - start_time:.4f} seconds")


Enter a query text in Kannada: ಸಂಗೀತ

Generated augmented response:
Query: 'ಸಂಗೀತ'
Combined Retrieved Documents:
Document: 'ಸಂಗೀತವು ಭಾವನೆಗಳನ್ನು ಮೂಡಿಸುತ್ತದೆ.' (Distance: 135.3819)
Document: 'ಬಾಹ್ಯಾಕಾಶದಲ್ಲಿ ವೈಜ್ಞಾನಿಕ ಮಹತ್ವದ ಹೆಜ್ಜೆಗಳು.' (Distance: 140.8398)
Document: 'ಸಂಗೀತವು ಭಾವನೆಗಳಿಗೆ ಜೀವ ಕೊಡುತ್ತದೆ.' (Distance: 146.2629)
Document: 'ಸಾರಿಗೆ ಸುರಕ್ಷತೆಯನ್ನು ಹೆಚ್ಚಿಸುತ್ತದೆ.' (Distance: 146.5990)
Document: 'ಕೃಷಿ ನಮ್ಮ ಜೀವಿತಾಧಾರ.' (Distance: 146.9215)
Augmented Response: 'Based on the retrieved documents, the information provided suggests: ಸಂಗೀತವು ಭಾವನೆಗಳನ್ನು ಮೂಡಿಸುತ್ತದೆ. ಬಾಹ್ಯಾಕಾಶದಲ್ಲಿ ವೈಜ್ಞಾನಿಕ ಮಹತ್ವದ ಹೆಜ್ಜೆಗಳು. ಸಂಗೀತವು ಭಾವನೆಗಳಿಗೆ ಜೀವ ಕೊಡುತ್ತದೆ. ಸಾರಿಗೆ ಸುರಕ್ಷತೆಯನ್ನು ಹೆಚ್ಚಿಸುತ್ತದೆ. ಕೃಷಿ ನಮ್ಮ ಜೀವಿತಾಧಾರ.'


Time taken for retrieval and generation: 0.0003 seconds
